In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

In [2]:
labevents     = pd.read_parquet(hosp+'labevents.parquet')
d_labitems    = pd.read_parquet(hosp+'d_labitems.parquet')

patients      = pd.read_parquet(hosp+'patients.parquet')
admissions    = pd.read_parquet(hosp+'admissions.parquet')

diagnoses_icd = pd.read_parquet(hosp+'diagnoses_icd.parquet')
microbiology  = pd.read_parquet(hosp+'microbiologyevents.parquet')
prescriptions = pd.read_parquet(hosp+'prescriptions.parquet')

In [3]:
chartevents     = pd.read_parquet(icu+'chartevents.parquet')
d_items         = pd.read_parquet(icu+'d_items.parquet')
inputevents     = pd.read_parquet(icu+'inputevents.parquet')
procedureevents = pd.read_parquet(icu+'procedureevents.parquet')
icustays        = pd.read_parquet(icu+'icustays.parquet')

In [4]:
# to_datetime
chartevents['charttime'] = pd.to_datetime(chartevents['charttime'])

# Demographic

### Gender

In [106]:
patients_gender = patients[['subject_id','gender']]
patients_gender.loc[patients_gender.gender == 'F' , 'gender'] = 1
patients_gender.loc[patients_gender.gender == 'M' , 'gender'] = 0

In [107]:
patients_gender

,subject_id,gender
0,10000032,1
1,10000048,1
2,10000068,1
3,10000084,0
4,10000102,1
...,...,...
299707,19999828,1
299708,19999829,1
299709,19999840,0
299710,19999914,1


### Age

In [108]:
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

In [109]:
icustays_intime = icustays[['subject_id','hadm_id','stay_id','intime']]
patients_age = patients[['subject_id','anchor_age','anchor_year']]

icustays_age = pd.merge(icustays_intime, patients_age, on = 'subject_id', how = 'left')

In [110]:
icustays_age['anchor_age_delta'] = pd.to_timedelta(icustays_age['anchor_age']*365.25, unit='D')
icustays_age['anchor_year'] = pd.to_datetime(icustays_age['anchor_year'],format="%Y")

icustays_age['delta'] = icustays_age['intime'] - icustays_age['anchor_year']
icustays_age['age'] = ((icustays_age['anchor_age_delta'] + icustays_age['delta'])/365.25).dt.days

icustays_age = icustays_age[['subject_id','hadm_id','stay_id','age']]

In [111]:
icustays_age

,subject_id,hadm_id,stay_id,age
0,10000032,29079034,39553978,52
1,10000980,26913865,39765666,76
2,10001217,24597018,37067082,55
3,10001217,27703517,34592300,55
4,10001725,25563031,31205490,46
...,...,...,...,...
73176,19999442,26785317,32336619,43
73177,19999625,25304202,31070865,82
73178,19999828,25744818,36075953,48
73179,19999840,21033226,38978960,58


### Race

In [57]:
admissions_race = admissions[['subject_id','race']].copy()

In [58]:
admissions_race['race'].replace(['ASIAN - ASIAN INDIAN', 'ASIAN - CHINESE','ASIAN - KOREAN', 'ASIAN - SOUTH EAST ASIAN'],'ASIAN',inplace=True)
admissions_race['race'].replace(['BLACK/AFRICAN AMERICAN','BLACK/AFRICAN','BLACK/CAPE VERDEAN','BLACK/CARIBBEAN ISLAND'],'BLACK',inplace=True)
admissions_race['race'].replace(['HISPANIC/LATINO - CENTRAL AMERICAN','HISPANIC/LATINO - COLUMBIAN','HISPANIC/LATINO - CUBAN','HISPANIC/LATINO - DOMINICAN','HISPANIC/LATINO - GUATEMALAN','HISPANIC/LATINO - HONDURAN','HISPANIC/LATINO - MEXICAN',
                             'HISPANIC/LATINO - PUERTO RICAN','HISPANIC/LATINO - SALVADORAN','PORTUGUESE','SOUTH AMERICAN'],'HISPANIC OR LATINO',inplace=True)
admissions_race['race'].replace(['NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER'],'PACIFIC ISLANDER',inplace=True)
admissions_race['race'].replace(['UNABLE TO OBTAIN','PATIENT DECLINED TO ANSWER'],'UNKNOWN',inplace=True)
admissions_race['race'].replace(['WHITE - BRAZILIAN','WHITE - EASTERN EUROPEAN','WHITE - OTHER EUROPEAN','WHITE - RUSSIAN'],'WHITE',inplace=True)

admissions_race = admissions_race.drop_duplicates()

In [61]:
multiple = admissions_race.subject_id.value_counts().loc[lambda x : x > 1].to_frame()
multiple.reset_index(inplace = True)
multiple = multiple.subject_id.unique()

hosp_race_multiple = admissions_race[admissions_race['subject_id'].isin(multiple)]

sol = []
multi = []
for i in hosp_race_multiple.subject_id.unique() :
    tmp = hosp_race_multiple[hosp_race_multiple['subject_id'] == i]
    if (tmp['race'] == 'UNKNOWN').any() :
        tmp = tmp[tmp['race'] != 'UNKNOWN']
    if len(tmp) <2 :
        sol.append(tmp)
    else : 
        multi.append(tmp)
sol = pd.concat(sol)
multi = pd.concat(multi)

multi['race'] = 'MULTIPLE RACE/ETHNICITY'
multi.drop_duplicates(inplace=True)

admissions_race = admissions_race[~admissions_race['subject_id'].isin(sol.subject_id.unique())]
admissions_race = admissions_race[~admissions_race['subject_id'].isin(multi.subject_id.unique())]

admissions_race = pd.concat([admissions_race,sol,multi])

In [62]:
admissions_race.race.unique()

array(['WHITE', 'OTHER', 'BLACK', 'UNKNOWN', 'HISPANIC OR LATINO',
       'ASIAN', 'PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKA NATIVE',
       'MULTIPLE RACE/ETHNICITY'], dtype=object)

# Lab values

In [5]:
labvalues = pd.read_csv('./data/labvalues/labvalues.csv')
labvalues

,labvalue,abbreviation,itemid,lb,ub,lb_cond,ub_cond
0,Albumin,Alb,227456,0.6,6.0,ge,le
1,Alkaline Phosphate,Alk_Phos,225612,20.0,3625.0,ge,le
2,Anion Gap,AG,227073,5.0,50.0,ge,le
3,Blood urea nitrogen,BUN,225624,0.0,300.0,gt,le
4,Calcium non-ionized,Ca,225625,0.2,20.0,ge,le
5,Creatine Kinase,CK,225634,0.0,30000.0,ge,le
6,Direct Bilirubin,D_Bil,225651,0.0,100.0,gt,le
7,Glucose,Glu,220621,20.0,2000.0,ge,le
8,Hematocrit,HCT,220545,0.0,100.0,gt,lt
9,International Normalized Ratio,INR,227467,0.3,10.0,ge,le


In [8]:
for i,idx in tqdm(enumerate(labvalues.abbreviation)):
    print("[%i/%i] Processing %s..."%(i+1,len(labvalues),labvalues.iloc[i].labvalue))
    globals()['chartevents_{}'.format(idx)] = chartevents[chartevents['itemid'].isin([labvalues.iloc[i].itemid])][['subject_id','hadm_id','stay_id','charttime','itemid','valuenum']]

    if labvalues.iloc[i].lb_cond == 'ge' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] >= labvalues.iloc[i].lb]
    elif labvalues.iloc[i].lb_cond == 'gt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] > labvalues.iloc[i].lb]

    if labvalues.iloc[i].ub_cond == 'le' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] <= labvalues.iloc[i].ub]
    elif labvalues.iloc[i].ub_cond == 'lt' : 
        globals()['chartevents_{}'.format(idx)] = globals()['chartevents_{}'.format(idx)][globals()['chartevents_{}'.format(idx)]['valuenum'] < labvalues.iloc[i].ub]

    globals()['chartevents_{}'.format(idx)].sort_values(by=['subject_id','charttime'],ascending=True,inplace=True)
    globals()['chartevents_{}'.format(idx)].reset_index(inplace=True, drop=True)

    if not os.path.isfile('./data/labvalues/chartevents_%s.parquet'%idx):
        globals()['chartevents_{}'.format(idx)].to_parquet('./data/labvalues/chartevents_%s.parquet'%idx)

0it [00:00, ?it/s]

[1/43] Processing Albumin...
[2/43] Processing Alkaline Phosphate...
[3/43] Processing Anion Gap...
[4/43] Processing Blood urea nitrogen...
[5/43] Processing Calcium non-ionized...
[6/43] Processing Creatine Kinase...
[7/43] Processing Direct Bilirubin...
[8/43] Processing Glucose...
[9/43] Processing Hematocrit...
[10/43] Processing International Normalized Ratio...
[11/43] Processing PH...
[12/43] Processing Phosphorous...
[13/43] Processing Platelet Count...
[14/43] Processing Serum chloride...
[15/43] Processing Serum creatinine...
[16/43] Processing Serum Sodium...
[17/43] Processing Serum Potassium...
[18/43] Processing Total Bilirubin...
[19/43] Processing White blood cell count...
[20/43] Processing Serum glucose...
[21/43] Processing Magnesium...
[22/43] Processing Ionized calcium...
[23/43] Processing Serum HCO3...
[24/43] Processing AST...
[25/43] Processing ALT...
[26/43] Processing PTT...
[27/43] Processing Arterial O2 pressure...
[28/43] Processing Arterial CO2 Pressure.

# Lab values in hosp

In [6]:
d_labitems

,itemid,label,fluid,category
0,50801,Alveolar-arterial Gradient,Blood,Blood Gas
1,50802,Base Excess,Blood,Blood Gas
2,50803,"Calculated Bicarbonate, Whole Blood",Blood,Blood Gas
3,50804,Calculated Total CO2,Blood,Blood Gas
4,50805,Carboxyhemoglobin,Blood,Blood Gas
...,...,...,...,...
1617,53150,Anti Hbs,Blood,Chemistry
1618,53151,Anti-la,Blood,Chemistry
1619,53152,HIV FINAL,Blood,Chemistry
1620,53153,HIV Screen,Blood,Chemistry


In [32]:
labevents[labevents['itemid'].isin([50821])]

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
5466,5476,10000935,25849114.0,42157155,50821,None,2187-10-22 15:40:00,2187-10-22 15:42:00,86,86.0,mm Hg,85.0,105.0,None,None,None
8314,8325,10000980,20897796.0,22563496,50821,None,2193-08-14 21:41:00,2193-08-14 21:43:00,___,30.0,mm Hg,85.0,105.0,abnormal,None,NO CALLS MADE - NOT ARTERIAL BLOOD.
15691,15709,10001884,29678536.0,68620790,50821,None,2130-10-10 09:31:00,2130-10-10 09:32:00,73,73.0,mm Hg,85.0,105.0,abnormal,None,None
15820,15838,10001884,NaN,17165845,50821,None,2130-10-19 13:58:00,2130-10-19 14:00:00,___,23.0,mm Hg,85.0,105.0,abnormal,None,NO CALLS MADE - NOT ARTERIAL BLOOD.
16069,16088,10001884,28664981.0,96023257,50821,None,2130-11-29 00:15:00,2130-11-29 00:17:00,103,103.0,mm Hg,85.0,105.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118171068,118352206,19999840,21033226.0,65766133,50821,None,2164-09-17 13:34:00,2164-09-17 13:36:00,___,25.0,mm Hg,85.0,105.0,abnormal,None,VERIFIED. PROVIDER NOTIFIED PER CURRENT LAB P...
118171081,118352219,19999840,21033226.0,2279408,50821,None,2164-09-17 13:39:00,2164-09-17 13:48:00,___,23.0,mm Hg,85.0,105.0,abnormal,None,PROVIDER NOTIFIED PER CURRENT LAB POLICY.
118171156,118352295,19999987,NaN,5532443,50821,None,2145-11-02 20:27:00,2145-11-02 20:29:00,439,439.0,mm Hg,85.0,105.0,abnormal,None,None
118171205,118352344,19999987,23865745.0,90055536,50821,None,2145-11-03 05:28:00,2145-11-03 05:31:00,114,114.0,mm Hg,85.0,105.0,abnormal,None,None


In [31]:
labevents[labevents['itemid'].isin([51006])]

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
38,39,10000032,NaN,52958335,51006,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,13,13.0,mg/dL,6.0,20.0,None,ROUTINE,None
107,108,10000032,NaN,43430170,51006,None,2180-05-06 22:25:00,2180-05-06 23:16:00,25,25.0,mg/dL,6.0,20.0,abnormal,STAT,None
155,156,10000032,22595853.0,93044861,51006,None,2180-05-07 05:05:00,2180-05-07 07:03:00,25,25.0,mg/dL,6.0,20.0,abnormal,ROUTINE,None
175,176,10000032,NaN,41498752,51006,P28Z0X,2180-06-03 12:00:00,2180-06-03 13:04:00,21,21.0,mg/dL,6.0,20.0,abnormal,ROUTINE,None
191,192,10000032,NaN,86302346,51006,P260SK,2180-06-03 12:00:00,2180-06-03 13:04:00,22,22.0,mg/dL,6.0,20.0,abnormal,ROUTINE,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118171282,118352421,19999987,23865745.0,9502935,51006,None,2145-11-05 06:10:00,2145-11-05 07:02:00,19,19.0,mg/dL,6.0,20.0,None,ROUTINE,None
118171319,118352458,19999987,23865745.0,88125093,51006,None,2145-11-06 10:07:00,2145-11-06 11:19:00,14,14.0,mg/dL,6.0,20.0,None,ROUTINE,None
118171339,118352478,19999987,23865745.0,80808239,51006,None,2145-11-07 06:00:00,2145-11-07 06:57:00,8,8.0,mg/dL,6.0,20.0,None,ROUTINE,None
118171354,118352493,19999987,23865745.0,22001078,51006,None,2145-11-09 05:30:00,2145-11-09 06:59:00,8,8.0,mg/dL,6.0,20.0,None,ROUTINE,None
